# Functions

In [1]:
#@title Traditional Collatz Map
def collatz(x):
  return int((3*x+1)*(x%2) + x/2*(1-(x%2)))

In [2]:
#@title Reduced Collatz Map
def reduced_collatz(x):
  return int((x + (2*x+1)*(x%2))/2)

In [3]:
#@title Modular Collatz Map
def mod_collatz(x,k,r):
  if x%2 == 1:
    r += 3**k
  if r%2 == 1:
    k += 1
    r = 3*r+1
  r = r//2
  x = x//2
  return x,k,r

def recover_value_mod(x,k,r):
  return x * 3**k + r

In [4]:
#@title Multi-register Collatz Map
def multi_mod_collatz(r, k, T, verbose=0):
  # r = [x,r_1, r_2, ...] are the registers for each base place
  # k = [k_0 - k_1, k_1 - k_2, ..., 0] are the differences in magnitude between each base place

  if len(r) < 2:
    r.append(0)
    k.append(0)
    if verbose > 0: print(r, k, "Created a new register")

  i = 0
  while i < len(r)-1:

    if r[i] % 2 == 1: # if this register is odd, move this bit to the next register using 3^corresponding k[i]
      r[i] -= 1
      r[i+1] += 3**k[i]
    r[i] //= 2 # and shift the zero out

    if verbose > 0: print(r, k, f"Collatz on index {i}")

    if r[i] == 0: # we can drop empty registers
      del r[i]
      if i > 0:
        k[i-1] += k[i] # the new difference in the previous register is equivalent to jumping both registers
      del k[i]
      if verbose > 0: print(r, k, f"Dropped register at {i}")
    else:
      i += 1


  if r == [1,0] and k == [0,0]: # break condition
    return [1], [0]

  if r[-1] > T: # the condition upon which we create a new register (lowest register exceeds the threshold T)
    r.append(0)
    k.append(0)
    if verbose > 0: print(r, k, "Created a new register")

    if r[-2] % 2 == 1: # if we created a new last register, we need to apply the "last" iteration here as well
      r[-2] -= 1
      r[-1] += 3**k[-2]
    r[-2] //= 2
    if verbose > 0: print(r, k, "Collatz on second to last register")

  if r[-1] % 2 == 1: # on the last register, though, perform 3x+1 and increment k[-2] if it is odd
    r[-1] = (3*r[-1] + 1)
    if len(k) > 1:
      k[-2] += 1
  r[-1] //= 2

  if verbose > 0: print(r, k, "Collatz on last register")

  return r,k

def recover_value_multi(r,k):
  value = 0
  for i in range(len(r)):
    value += r[i]
    value *= 3**k[i]
  return value

In [5]:
#@title Base Collatz Number
class BaseCollatz:
  def __init__(self, x=1):
    self.tail = [2]
    self.expression = [0]
    self.assign(x)

  def assign(self, x): # run Collatz to find decomposition
    while x != 1:
      if x%2 == 1:
        x = 3*x+1
        self.expression.append(0)
      if x == 1: break
      x //= 2
      self.expression[-1] += 1

  def value(self): # run Collatz to recover value (the reverse Collatz map)
    d, k, r, y = 0, 0, 0, 0
    x = self.expression

    # Loop until x contains terms
    while len(x) > 0:
      # Loop while x has 0 in the least significant place
      while len(x) > 0 and x[0] == 0:
        if r % 2 == 0:
          y += 2**d # set y_d to 1
          r += 3**k
        r = 3*r + 1
        k += 1
        del x[0]  # Shift out the least significant place of x (the zero)

      # If r is odd, add 1 to the beginning of the result
      if r % 2 == 1:
        y += 2**d # set y_d to 1
        r += 3**k

      d += 1
      r //= 2
      if len(x) > 0:
        x[0] -= 1  # Decrement the least significant term of x

    return y

In [6]:
#@title Compute Diagram Branches
# returns [stay, increment] r values
# k stays the same if x and r are same parity else it increments
# on making each step, drop the last bit of x
# x bar moves toward 0 if x even, else toward -1
def branches(x,k,r):
  if r%2 == 0:
    stay = r//2 # x even; move toward 0
    up = (3*r + 3**(k+1) + 1) // 2 # x odd; move toward -1
  else:
    stay = (r + 3**(k))//2 # x odd; move toward -1
    up = (3*r + 1) // 2 # x even; move toward 0
  return [stay, up]

def x_bar(k,r):
  return -r/3**k

# Tests

In [7]:
#@title Test Collatz
print("x, Collatz, reduced if 3x+1")
for x in range(1,10):
  print(f"{x}: {collatz(x)}, {reduced_collatz(x)}")



x, Collatz, reduced if 3x+1
1: 4, 2
2: 1, 1
3: 10, 5
4: 2, 2
5: 16, 8
6: 3, 3
7: 22, 11
8: 4, 4
9: 28, 14


In [8]:
#@title Test Reduced Collatz
for x in range(1, 30):
  print()
  print(x, end='-')
  while x != 1:
    x = reduced_collatz(x)
    print(x, end='-')


1-
2-1-
3-5-8-4-2-1-
4-2-1-
5-8-4-2-1-
6-3-5-8-4-2-1-
7-11-17-26-13-20-10-5-8-4-2-1-
8-4-2-1-
9-14-7-11-17-26-13-20-10-5-8-4-2-1-
10-5-8-4-2-1-
11-17-26-13-20-10-5-8-4-2-1-
12-6-3-5-8-4-2-1-
13-20-10-5-8-4-2-1-
14-7-11-17-26-13-20-10-5-8-4-2-1-
15-23-35-53-80-40-20-10-5-8-4-2-1-
16-8-4-2-1-
17-26-13-20-10-5-8-4-2-1-
18-9-14-7-11-17-26-13-20-10-5-8-4-2-1-
19-29-44-22-11-17-26-13-20-10-5-8-4-2-1-
20-10-5-8-4-2-1-
21-32-16-8-4-2-1-
22-11-17-26-13-20-10-5-8-4-2-1-
23-35-53-80-40-20-10-5-8-4-2-1-
24-12-6-3-5-8-4-2-1-
25-38-19-29-44-22-11-17-26-13-20-10-5-8-4-2-1-
26-13-20-10-5-8-4-2-1-
27-41-62-31-47-71-107-161-242-121-182-91-137-206-103-155-233-350-175-263-395-593-890-445-668-334-167-251-377-566-283-425-638-319-479-719-1079-1619-2429-3644-1822-911-1367-2051-3077-4616-2308-1154-577-866-433-650-325-488-244-122-61-92-46-23-35-53-80-40-20-10-5-8-4-2-1-
28-14-7-11-17-26-13-20-10-5-8-4-2-1-
29-44-22-11-17-26-13-20-10-5-8-4-2-1-

In [9]:
#@title Test Modular Collatz
x = 17
d, k, r = 0, 0, 0
print("x_d d   k   r   value")
while x != 0 or r != 1:
  print(x%2," ",d," ",k," ",r, " ", recover_value_mod(x,k,r))
  x,k,r = mod_collatz(x,k,r)
  d += 1
print(x%2," ",d," ",k," ",r, " ", recover_value_mod(x,k,r))

x_d d   k   r   value
1   0   0   0   17
0   1   1   2   26
0   2   1   1   13
0   3   2   2   20
1   4   2   1   10
0   5   2   5   5
0   6   3   8   8
0   7   3   4   4
0   8   3   2   2
0   9   3   1   1


In [10]:
#@title Test Multi Modular Collatz
r = [218686112] # r[0] is x
k = [0]
verbose = 1

print("[x, r_n,...] [k_n, ...]")

# add the collatz base expression as a list of numbers, implement methods for iterating collatz and algorithm 4
while r != [1]:
  print(recover_value_multi(r,k), end='\n' if verbose else '-')
  r,k = multi_mod_collatz(r,k,27, verbose=verbose)
print(recover_value_multi(r,k))

[x, r_n,...] [k_n, ...]
218686112
[218686112, 0] [0, 0] Created a new register
[109343056, 0] [0, 0] Collatz on index 0
[109343056, 0] [0, 0] Collatz on last register
109343056
[54671528, 0] [0, 0] Collatz on index 0
[54671528, 0] [0, 0] Collatz on last register
54671528
[27335764, 0] [0, 0] Collatz on index 0
[27335764, 0] [0, 0] Collatz on last register
27335764
[13667882, 0] [0, 0] Collatz on index 0
[13667882, 0] [0, 0] Collatz on last register
13667882
[6833941, 0] [0, 0] Collatz on index 0
[6833941, 0] [0, 0] Collatz on last register
6833941
[3416970, 1] [0, 0] Collatz on index 0
[3416970, 2] [1, 0] Collatz on last register
10250912
[1708485, 2] [1, 0] Collatz on index 0
[1708485, 1] [1, 0] Collatz on last register
5125456
[854242, 4] [1, 0] Collatz on index 0
[854242, 2] [1, 0] Collatz on last register
2562728
[427121, 2] [1, 0] Collatz on index 0
[427121, 1] [1, 0] Collatz on last register
1281364
[213560, 4] [1, 0] Collatz on index 0
[213560, 2] [1, 0] Collatz on last register

In [11]:
#@title Test Base Collatz Object, includiong Reverse Collatz Map
number = BaseCollatz(5) # should be [0,4]
print(number.expression)
print(number.value())

number = BaseCollatz(17) # should be [0,2,3,4]
print(number.expression)
print(number.value())

number = BaseCollatz(218686112) # should be [5, 6, 9, 9, 1, 2, 3, 4]
print(number.expression)
print(number.value())

[0, 4]
5
[0, 2, 3, 4]
17
[5, 6, 9, 9, 1, 2, 3, 4]
218686112


In [12]:
#@title Test the Diagram Branches
x = 15
d, k, r = 0, 0, 0
print("k   r   options  x bar")
for i in range(16):
  print(k," ",r, " ", branches(x,k,r), " ", x_bar(k,r))
  if x == 0 and r == 1: print()
  x,k,r = mod_collatz(x,k,r)
  d += 1
print(k," ",r, " ", branches(x,k,r), " ", x_bar(k,r))

k   r   options  x bar
0   0   [0, 2]   0.0
1   2   [1, 8]   -0.6666666666666666
2   8   [4, 26]   -0.8888888888888888
3   26   [13, 80]   -0.9629629629629629
4   80   [40, 242]   -0.9876543209876543
4   40   [20, 182]   -0.49382716049382713
4   20   [10, 152]   -0.24691358024691357
4   10   [5, 137]   -0.12345679012345678
4   5   [43, 8]   -0.06172839506172839
5   8   [4, 377]   -0.03292181069958848
5   4   [2, 371]   -0.01646090534979424
5   2   [1, 368]   -0.00823045267489712
5   1   [122, 2]   -0.00411522633744856

6   2   [1, 1097]   -0.0027434842249657062
6   1   [365, 2]   -0.0013717421124828531

7   2   [1, 3284]   -0.0009144947416552355
7   1   [1094, 2]   -0.0004572473708276177


Interpretation:
* if k increases, r becomes the second option, else the first,
and x bar is the rational number which would remain in layer k forever starting at r
* notice how x bar tends to zero once we run out of digits
* at this point, x bar roughly halves each iteration, and x bar chases -1/3**k; once they meet, the 1-4-2-1 cycle starts
* the halving at each step means that, in some sense, it is converging to zero (consider x bar to be some measure of the process)